In [1]:
import pandas as pd
from itertools import combinations

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor

import sys
sys.path.append("..")
from meta_stream import MetaStream

In [2]:
df = pd.read_csv("../../data/electricity-normalized.csv")
df = df[['period', 'nswprice', 'nswdemand', 'vicprice', 'vicdemand', 'transfer']]

In [3]:
# NOTE: list of regression algorithms
models =    [
            SVR(),
            RandomForestRegressor(random_state=42),
            LinearRegression(),
            Ridge(),
            GradientBoostingRegressor(random_state=42),
            DecisionTreeRegressor()
            ]

Parameter settings for MetaStream (considering a meta-level window size of 200)

In [4]:
base_data_window = 96*2
base_delay_window = 0
base_sel_window_size = 48*2
meta_data_window = 200

In [5]:
meta_learner = RandomForestClassifier()
for pair in combinations(models, 2):
    print(f'Regressor pair: {type(pair[0]).__name__}, {type(pair[1]).__name__}')
    
    metas = MetaStream(meta_learner, pair, base_data_window, base_delay_window, base_sel_window_size, meta_data_window, strategy='tie', threshold=.05)
    metas.base_train(data=df, target='nswdemand')
    metas.meta_train(data=df, target='nswdemand', default=True, ensemble=True, report=True)
    
    print('######################################')

Regressor pair: SVR, RandomForestRegressor
Mean score Recommended 0.544+-0.571
Meta-level score Recommended 0.641
Mean score default 0.338+-0.373
Meta-level score default 0.219
Mean score ensemble 0.420+-0.429
Mean score reg 1. 0.694
Mean score reg 2. 0.338
######################################
Regressor pair: SVR, LinearRegression
Mean score Recommended 0.466+-0.397
Meta-level score Recommended 0.493
Mean score default 0.694+-0.689
Meta-level score default 0.596
Mean score ensemble 0.427+-0.349
Mean score reg 1. 0.694
Mean score reg 2. 0.429
######################################
Regressor pair: SVR, Ridge
Mean score Recommended 0.682+-0.624
Meta-level score Recommended 0.541
Mean score default 0.694+-0.689
Meta-level score default 0.459
Mean score ensemble 0.669+-0.687
Mean score reg 1. 0.694
Mean score reg 2. 0.752
######################################
Regressor pair: SVR, GradientBoostingRegressor
Mean score Recommended 0.518+-0.498
Meta-level score Recommended 0.615
Mean score d